<a href="https://colab.research.google.com/github/quotation3/TIL/blob/master/topic_modeling_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas as pd
import numpy as np
from sklearn.decomposition import randomized_svd

In [73]:
class LSA:

    def __init__(self, text):
        self.text = text
        self.tokens = []
        self.tdm = []

    def make_tdm(self):

        text_word = []
        for i in range(len(text)):
            text_word.append(text[i].split(' '))    # 각 텍스트들의 토큰 추출
            self.tokens.extend(text[i].split(' '))
        self.tokens = list(set(self.tokens))    # 전체 텍스트의 토큰 추출

# 전체 텍스트의 토큰들이 각 텍스트들의 토큰과 맞는지 비교하여 맞는 만큼 cnt_list에 플러스
        for i in range(len(text_word)):
            cnt_list = [0 for a in range(len(self.tokens))]
            for j in range(len(self.tokens)):
                if self.tokens[j] in text_word[i]:
                    cnt_list[j] += text_word[i].count(self.tokens[j])
            self.tdm.append(cnt_list)
        self.tdm

        df=pd.DataFrame(self.tdm, index=self.text, columns=self.tokens)
        
        return df

    def topic_rank(self, topic_num, word_num):

 # 특이값 분해
        U, s, VT = randomized_svd(np.array(self.tdm), n_components=4, n_iter=10, random_state=None)
        vt = VT[:topic_num, :]  # 토픽 수만큼 슬라이싱

        column_name = ['topic{}'.format(i+1) for i in range(topic_num)]
        df1 = pd.DataFrame(vt.T, index=self.tokens, columns=column_name)    # topic의 수만큼 열로 만든 데이터프레임 생성
        
        topic_result = []
        for name in column_name:
            topic_top = df1[name].sort_values(ascending=False)[0:word_num]  # 데이터 프레임을 열별로 내림차순하여 상위 3개를 추출
            topic_result.append(([topic_top.index, topic_top.values]))

        rank_by_topic =[]
        for i in range(len(topic_result)):
            tmp = []
            for j in range(word_num):
                tmp.append([topic_result[i][0][j], topic_result[i][1][j]])
            rank_by_topic.append(tmp)

        result = []
        for i in range(len(rank_by_topic)):
            result.append([column_name[i], rank_by_topic[i]])
            print('{}:{}'.format(result[i][0],result[i][1]))

In [74]:
text = ['바나나 사과 포도 포도 짜장면','사과 포도','포도 바나나','짜장면 짬뽕 탕수육','볶음밥 탕수육','짜장면 짬뽕','라면 스시','스시 짜장면','가츠동 스시 소바','된장찌개 김치찌개 김치','김치 된장 짜장면','비빔밥 김치']
topic_num = 4
word_num = 3

In [75]:
lsa = LSA(text)

In [76]:
lsa.make_tdm()

,된장,바나나,짜장면,소바,포도,스시,김치찌개,사과,볶음밥,라면,김치,비빔밥,가츠동,짬뽕,된장찌개,탕수육
바나나 사과 포도 포도 짜장면,0,1,1,0,2,0,0,1,0,0,0,0,0,0,0,0
사과 포도,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
포도 바나나,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
짜장면 짬뽕 탕수육,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1
볶음밥 탕수육,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
짜장면 짬뽕,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
라면 스시,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
스시 짜장면,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
가츠동 스시 소바,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0
된장찌개 김치찌개 김치,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0


In [77]:
lsa.topic_rank(topic_num,word_num)

topic1:[['포도', 0.6968625387988908], ['짜장면', 0.48563448882209026], ['사과', 0.3484312693994454]]
topic2:[['짜장면', 0.5841558837852063], ['짬뽕', 0.3555556968187594], ['김치', 0.33700147191297775]]
topic3:[['김치', 0.610903527729413], ['된장찌개', 0.264397849195179], ['김치찌개', 0.264397849195179]]
topic4:[['스시', 0.5523884496346754], ['김치', 0.37136541740505563], ['소바', 0.2766984641841558]]
